In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import os
import numpy as np
from datetime import datetime, timedelta

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [37]:
def preprocess_image(image):
    # Convert to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Histogram Equalization to improve contrast
    # image = cv2.equalizeHist(image)

    # Apply Gaussian Blur to reduce noise
    # blurred_image = cv2.GaussianBlur(equalized_image, (5, 5), 0)
    
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    image = clahe.apply(image)
    
    def adjust_gamma(image, gamma=1.0):
        invGamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(image, table)

    image = adjust_gamma(image, gamma=1.2)

    return image

In [38]:
# orb = cv2.ORB_create()
sift = cv2.SIFT_create()

# Load facebank
facebank_path = "facebank"
facebank = {}
facebank_descriptors = []

def preprocess_facebank(facebank_path):
    for file in os.listdir(facebank_path):
        filepath = os.path.join(facebank_path, file)
        if file.endswith(('png', 'jpg', 'jpeg')):
            label = os.path.splitext(file)[0]  # Use filename (without extension) as label
            image = cv2.imread(filepath)
            image = preprocess_image(image)
            keypoints, descriptors = sift.detectAndCompute(image, None)
            if descriptors is not None:
                facebank[label] = (keypoints, descriptors)
                facebank_descriptors.append(descriptors)
preprocess_facebank(facebank_path)

In [39]:
# Index parameters for KDTree (recommended for SIFT)
index_params = dict(algorithm=1,  # FLANN_INDEX_KDTREE for floating-point descriptors
                    trees=5)  # Number of trees in the KDTree

search_params = dict(checks=50)  # Number of checks during the search

flann = cv2.FlannBasedMatcher(index_params, search_params)
# flann = cv2.FlannBasedMatcher(dict(algorithm=1), dict(checks=50))

def match_face(input_descriptors):
    best_match = None
    best_score = float('inf')
    
    input_descriptors = np.float32(input_descriptors)
    
    for label, (_, descriptors) in facebank.items():
        # Perform KNN matching (K=2, to get the two best matches)
        
        descriptors = np.float32(descriptors) 

        matches = flann.knnMatch(input_descriptors, descriptors, k=2)

        # Apply ratio test (Lowe's ratio test) to filter good matches
        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:  # Ensure we have two matches to unpack
                m, n = match_pair  # Unpack the two closest matches
                if m.distance < 0.7 * n.distance:  # 0.7 is a commonly used threshold for ratio test
                    good_matches.append(m)

        score = sum([match.distance for match in good_matches]) / len(good_matches) if good_matches else float('inf')

        if score < best_score:
            best_match = label
            best_score = score
    print(best_score)

    return best_match if best_score < 120 else "Unknown"  # Threshold for matching


In [40]:
presence_log = {}

def update_presence_log(name):
    now = datetime.now()
    if name in presence_log:
        presence_log[name]["last_seen"] = now
    else:
        presence_log[name] = {"start_time": now, "last_seen": now}
        
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = preprocess_image(frame)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(processed_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Process each detected face
    for (x, y, w, h) in faces:
        face = processed_frame[y:y+h, x:x+w]
        cv2.imshow("Detected Face", face)
        
        keypoints, descriptors = sift.detectAndCompute(face, None)
        
        if descriptors is not None:
            name = match_face(descriptors)
            update_presence_log(name)
            
            # Draw rectangle around face and label
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display presence log
for person, times in presence_log.items():
    duration = times["last_seen"] - times["start_time"]
    print(f"{person}: Present for {duration}")

inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
Unknown: Present for 0:00:03.393896


_______

In [48]:
import cv2
import numpy as np
import os
from datetime import datetime

# Preprocess image
def preprocess_image(image):
    # Convert to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE for better contrast
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    image = clahe.apply(image)

    # Adjust gamma to normalize brightness
    def adjust_gamma(image, gamma=1.5):
        invGamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(image, table)

    image = adjust_gamma(image)
    return image

# Initialize SIFT
sift = cv2.SIFT_create()

# Load facebank
facebank_path = "facebank"
facebank = {}

def preprocess_facebank(facebank_path):
    for idx, person_folder in enumerate(os.listdir(facebank_path)):
        person_folder_path = os.path.join(facebank_path, person_folder)
        if os.path.isdir(person_folder_path):  # Check if it's a folder
            for file_idx, file in enumerate(os.listdir(person_folder_path)):
                filepath = os.path.join(person_folder_path, file)
                if file.endswith(('png', 'jpg', 'jpeg')):
                    label = person_folder  # Folder name as the label
                    print(f"[{idx}.{file_idx}] Processing {label}: {file}")
                    image = cv2.imread(filepath)
                    image = preprocess_image(image)
                    resized_face = cv2.resize(image, (150, 150))  # Normalize face size
                    keypoints, descriptors = sift.detectAndCompute(resized_face, None)
                    if descriptors is not None:
                        if label not in facebank:
                            facebank[label] = []
                        facebank[label].append(descriptors)
                        # facebank[label] = (keypoints, descriptors)
                        # facebank_descriptors.append(descriptors)
preprocess_facebank(facebank_path)

# FLANN Matcher
index_params = dict(algorithm=1, trees=5)  # KDTree for SIFT
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

def match_face(input_descriptors):
    best_match = None
    best_score = float('inf')

    input_descriptors = np.float32(input_descriptors)
    for label, descriptors_list in facebank.items():
        for descriptors in descriptors_list:
            descriptors = np.float32(descriptors)
            matches = flann.knnMatch(input_descriptors, descriptors, k=2)

            # Apply Lowe's ratio test
            good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]

            if good_matches:
                score = sum([m.distance for m in good_matches]) / len(good_matches)
                if score < best_score:
                    best_match = label
                    best_score = score
                
            print(len(good_matches))

    return best_match if best_score < 150 else "Unknown"

# Log presence
presence_log = {}

def update_presence_log(name):
    now = datetime.now()
    if name in presence_log:
        presence_log[name]["last_seen"] = now
    else:
        presence_log[name] = {"start_time": now, "last_seen": now}

# Initialize face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Stabilize recognition
last_recognized = {"name": "Unknown", "count": 0}

def match_face_with_lock(descriptors):
    global last_recognized
    name = match_face(descriptors)

    if name == last_recognized["name"]:
        last_recognized["count"] += 1
    else:
        last_recognized["count"] = 0
    last_recognized["name"] = name

    if last_recognized["count"] >= 3:
        return last_recognized["name"]
    return "Unknown"

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = preprocess_image(frame)

    # Detect faces
    faces = face_cascade.detectMultiScale(processed_frame, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
    
    for (x, y, w, h) in faces:
        face = processed_frame[y:y+h, x:x+w]
        
        # Edge detection to filter blurred faces
        edges = cv2.Canny(face, 100, 200)
        if np.sum(edges) < 10000:
            print("Wajah terlalu buram, abaikan.")
            continue

        resized_face = cv2.resize(face, (150, 150))  # Normalize face size
        keypoints, descriptors = sift.detectAndCompute(resized_face, None)
        if descriptors is not None and len(descriptors) > 20:
            name = match_face_with_lock(descriptors)
            update_presence_log(name)

            # Draw rectangle and label
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display presence log
for person, times in presence_log.items():
    duration = times["last_seen"] - times["start_time"]
    print(f"{person}: Present for {duration}")


[0.0] Processing ANDREAS: ANDREAS.jpg
[0.1] Processing ANDREAS: ANDREAS.png
[1.0] Processing Bryan Eugene: Bryan Eugene.jpeg
[1.1] Processing Bryan Eugene: WIN_20241207_19_13_38_Pro.jpg
[1.2] Processing Bryan Eugene: WIN_20241207_19_13_40_Pro.jpg
[1.3] Processing Bryan Eugene: WIN_20241207_19_13_42_Pro.jpg
[1.4] Processing Bryan Eugene: WIN_20241207_19_13_44_Pro.jpg
[1.5] Processing Bryan Eugene: WIN_20241207_19_13_45_Pro.jpg
[2.0] Processing ENRICO: ENRICO.jpg
1
1
1
1
0
0
1
0
1
0
1
3
0
0
1
2
0
1
1
1
5
0
1
2
1
0
0
0
1
3
0
1
3
0
0
3
1
1
4
0
1
1
1
0
0
2
0
1
0
1
0
0
0
1
3
0
0
0
1
1
2
0
0
1
0
0
0
0
0
2
1
2
2
0
2
0
1
0
0
1
0
1
0
3
0
1
0
0
0
0
2
0
3
1
1
0
2
1
0
1
0
3
1
1
0
1
2
0
0
0
4
1
1
0
0
2
1
0
1
4
1
1
0
2
2
1
0
1
5
0
2
0
1
0
1
0
1
2
1
3
1
3
1
0
1
0
1
0
0
0
0
0
0
0
0
4
0
0
1
1
0
1
1
0
2
1
1
1
1
1
1
0
0
2
2
1
1
3
3
0
0
0
3
0
0
1
1
1
1
0
1
0
1
1
1
2
1
2
0
0
2
0
0
0
0
0
2
1
0
4
0
0
0
2
0
0
1
1
1
0
0
1
1
0
2
0
0
1
0
1
2
1
0
0
1
1
0
0
1
1
0
0
0
0
0
3
0
1
0
0
0
1
0
0
3
1
0
2
1
1
0
0
0
4
0
3
1


____________

# Hasil Akhir
Mengganti model untuk Matchernya dari FLANN menjadi BFMatcher

In [54]:
import cv2
import numpy as np
import os
from datetime import datetime

# Preprocess image
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    image = clahe.apply(image)

    def adjust_gamma(image, gamma=1.5):
        invGamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(image, table)

    image = adjust_gamma(image)
    return image

# Initialize SIFT
sift = cv2.SIFT_create()

# Load facebank
facebank_path = "facebank"
facebank = {}

def preprocess_facebank(facebank_path):
    for idx, person_folder in enumerate(os.listdir(facebank_path)):
        person_folder_path = os.path.join(facebank_path, person_folder)
        if os.path.isdir(person_folder_path):
            for file_idx, file in enumerate(os.listdir(person_folder_path)):
                filepath = os.path.join(person_folder_path, file)
                if file.endswith(('png', 'jpg', 'jpeg')):
                    label = person_folder
                    print(f"[{idx}.{file_idx}] Processing {label}: {file}")
                    image = cv2.imread(filepath)
                    image = preprocess_image(image)
                    resized_face = cv2.resize(image, (150, 150))
                    keypoints, descriptors = sift.detectAndCompute(resized_face, None)
                    if descriptors is not None:
                        if label not in facebank:
                            facebank[label] = []
                        facebank[label].append(descriptors)
preprocess_facebank(facebank_path)

# BFMatcher
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

def match_face(input_descriptors):
    best_match = None
    best_score = float('0')

    input_descriptors = np.float32(input_descriptors)
    for label, descriptors_list in facebank.items():
        for descriptors in descriptors_list:
            descriptors = np.float32(descriptors)
            matches = bf.knnMatch(input_descriptors, descriptors, k=2)

            # Apply Lowe's ratio test
            good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

            if good_matches:
                # score = sum([m.distance for m in good_matches]) / len(good_matches)
                score = len(good_matches)
                
                if score > best_score:
                    best_match = label
                    best_score = score

            print(f"Matches for {label}: {len(good_matches)}")

    return best_match 

# Log presence
presence_log = {}

def update_presence_log(name):
    now = datetime.now()
    if name in presence_log:
        presence_log[name]["last_seen"] = now
    else:
        presence_log[name] = {"start_time": now, "last_seen": now}

# Initialize face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Stabilize recognition
last_recognized = {"name": "Unknown", "count": 0}

def match_face_with_lock(descriptors):
    global last_recognized
    name = match_face(descriptors)

    if name == last_recognized["name"]:
        last_recognized["count"] += 1
    else:
        last_recognized["count"] = 0
    last_recognized["name"] = name

    if last_recognized["count"] >= 3:
        return last_recognized["name"]
    return "Unknown"

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = preprocess_image(frame)

    # Detect faces
    faces = face_cascade.detectMultiScale(processed_frame, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
    
    for (x, y, w, h) in faces:
        face = processed_frame[y:y+h, x:x+w]

        edges = cv2.Canny(face, 100, 200)
        if np.sum(edges) < 10000:
            print("Wajah terlalu buram, abaikan.")
            continue

        resized_face = cv2.resize(face, (150, 150))
        keypoints, descriptors = sift.detectAndCompute(resized_face, None)
        if descriptors is not None and len(descriptors) > 20:
            name = match_face_with_lock(descriptors)
            update_presence_log(name)

            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display presence log
for person, times in presence_log.items():
    duration = times["last_seen"] - times["start_time"]
    print(f"{person}: Present for {duration}")


[0.0] Processing ANDREAS: ANDREAS.jpg
[0.1] Processing ANDREAS: ANDREAS.png
[1.0] Processing Bryan Eugene: Bryan Eugene.jpeg
[1.1] Processing Bryan Eugene: WIN_20241207_19_13_38_Pro.jpg
[1.2] Processing Bryan Eugene: WIN_20241207_19_13_40_Pro.jpg
[1.3] Processing Bryan Eugene: WIN_20241207_19_13_42_Pro.jpg
[1.4] Processing Bryan Eugene: WIN_20241207_19_13_44_Pro.jpg
[1.5] Processing Bryan Eugene: WIN_20241207_19_13_45_Pro.jpg
[2.0] Processing ENRICO: ENRICO.jpg
Matches for ANDREAS: 2
Matches for ANDREAS: 5
Matches for Bryan Eugene: 2
Matches for Bryan Eugene: 4
Matches for Bryan Eugene: 2
Matches for Bryan Eugene: 3
Matches for Bryan Eugene: 1
Matches for Bryan Eugene: 3
Matches for ENRICO: 1
Matches for ANDREAS: 1
Matches for ANDREAS: 1
Matches for Bryan Eugene: 3
Matches for Bryan Eugene: 0
Matches for Bryan Eugene: 0
Matches for Bryan Eugene: 2
Matches for Bryan Eugene: 0
Matches for Bryan Eugene: 0
Matches for ENRICO: 1
Matches for ANDREAS: 2
Matches for ANDREAS: 6
Matches for Brya

____

In [ ]:
import cv2
import numpy as np
import os
from datetime import datetime

# Preprocess image
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    image = clahe.apply(image)

    def adjust_gamma(image, gamma=1.5):
        invGamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(image, table)

    image = adjust_gamma(image)
    return image

# Initialize SIFT
sift = cv2.SIFT_create()

# Load facebank
facebank_path = "facebank"
facebank = {}

def preprocess_facebank(facebank_path):
    for idx, person_folder in enumerate(os.listdir(facebank_path)):
        person_folder_path = os.path.join(facebank_path, person_folder)
        if os.path.isdir(person_folder_path):
            for file_idx, file in enumerate(os.listdir(person_folder_path)):
                filepath = os.path.join(person_folder_path, file)
                if file.endswith(('png', 'jpg', 'jpeg')):
                    label = person_folder
                    print(f"[{idx}.{file_idx}] Processing {label}: {file}")
                    image = cv2.imread(filepath)
                    image = preprocess_image(image)
                    resized_face = cv2.resize(image, (150, 150))
                    keypoints, descriptors = sift.detectAndCompute(resized_face, None)
                    if descriptors is not None:
                        if label not in facebank:
                            facebank[label] = []
                        facebank[label].append(descriptors)
preprocess_facebank(facebank_path)

# FLANN Matcher
FLANN_INDEX_KDTREE = 1
index_params = {"algorithm": FLANN_INDEX_KDTREE, "trees": 5}
search_params = {"checks": 50}
flann = cv2.FlannBasedMatcher(index_params, search_params)

def match_face(input_descriptors):
    best_match = None
    best_score = float('-inf')  # Score should be initialized for maximizing comparison.
    second_best_score = float('-inf')

    input_descriptors = cv2.normalize(input_descriptors, None, norm_type=cv2.NORM_L2)
    for label, descriptors_list in facebank.items():
        for descriptors in descriptors_list:
            descriptors = cv2.normalize(descriptors, None, norm_type=cv2.NORM_L2)
            matches = flann.knnMatch(input_descriptors, descriptors, k=2)

            # Apply Lowe's ratio test
            good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

            if good_matches:
                score = len(good_matches)
                if score > best_score:
                    second_best_score = best_score
                    best_score = score
                    best_match = label
                elif score > second_best_score:
                    second_best_score = score

    # Validate ratio between best and second-best scores
    if best_score / (second_best_score + 1e-5) < 1.2:
        return "Unknown"

    return best_match


# Log presence
presence_log = {}

def update_presence_log(name):
    now = datetime.now()
    if name in presence_log:
        presence_log[name]["last_seen"] = now
    else:
        presence_log[name] = {"start_time": now, "last_seen": now}

# Initialize face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Stabilize recognition
last_recognized_window = []

def match_face_with_lock(descriptors):
    global last_recognized_window

    name = match_face(descriptors)
    last_recognized_window.append(name)
    
    # Keep only the last 5 results
    if len(last_recognized_window) > 5:
        last_recognized_window.pop(0)
    
    # Determine the most common label
    most_common = max(set(last_recognized_window), key=last_recognized_window.count)
    if last_recognized_window.count(most_common) >= 3:
        return most_common
    return "Unknown"

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = preprocess_image(frame)

    # Detect faces
    faces = face_cascade.detectMultiScale(processed_frame, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
    
    for (x, y, w, h) in faces:
        margin = 10
        face = processed_frame[max(0, y-margin):min(y+h+margin, processed_frame.shape[0]), max(0, x-margin):min(x+w+margin, processed_frame.shape[1])]

        edges = cv2.Canny(face, 100, 200)
        if np.sum(edges) < 10000:
            print("Wajah terlalu buram, abaikan.")
            continue

        resized_face = cv2.resize(face, (150, 150))
        keypoints, descriptors = sift.detectAndCompute(resized_face, None)
        if descriptors is not None and len(descriptors) > 20:
            name = match_face_with_lock(descriptors)
            update_presence_log(name)

            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display presence log
for person, times in presence_log.items():
    duration = times["last_seen"] - times["start_time"]
    print(f"{person}: Present for {duration}")


[0.0] Processing ANDREAS: ANDREAS.jpg
[0.1] Processing ANDREAS: ANDREAS.png
[1.0] Processing Bryan Eugene: Bryan Eugene.jpeg
[1.1] Processing Bryan Eugene: WIN_20241207_19_13_38_Pro.jpg
[1.2] Processing Bryan Eugene: WIN_20241207_19_13_40_Pro.jpg
[1.3] Processing Bryan Eugene: WIN_20241207_19_13_42_Pro.jpg
[1.4] Processing Bryan Eugene: WIN_20241207_19_13_44_Pro.jpg
[1.5] Processing Bryan Eugene: WIN_20241207_19_13_45_Pro.jpg
[2.0] Processing ENRICO: ENRICO.jpg
[3.0] Processing facebank: ANDREAS VALENTINO.jpg
[3.1] Processing facebank: ARETHA.jpg
[3.2] Processing facebank: ARIF.jpg
[3.3] Processing facebank: AUDRY.jpg
[3.4] Processing facebank: AXEL.jpg
[3.5] Processing facebank: CHRISTIAN.jpg
[3.6] Processing facebank: EDRICK.jpg
[3.7] Processing facebank: GABRIEL.jpg
[3.8] Processing facebank: GUSTI.jpg
[3.9] Processing facebank: JAMES.jpg
[3.10] Processing facebank: JESSICA.jpg
[3.11] Processing facebank: JONATHAN.jpg
[3.12] Processing facebank: LUCKY .jpg
[3.13] Processing facebank